<img src="./images/paleoCAMPLogo.png"
         width="16%"
         alt="paleoCAMP_logo"
         align="right"
         style="horizontal-align:top;margin:0px 0px"/>

# Running CESM in Four Steps - Introduction

**Tutorial at [the 2024 paleoCAMP](https://paleoclimate.camp/) | June 18–July 1, 2024**  
       
[Jiang Zhu](https://staff.cgd.ucar.edu/jiangzhu/)  
[jiangzhu@ucar.edu](mailto:jiangzhu@ucar.edu)  
Climate & Global Dynamics Laboratory  
NSF National Center for Atmospheric Research  

------

**Learning Objectives:**

* Know basic structure of CESM as a software including the workspaces
* Know how to set up a simple experiment with CESM
* Know how to configure details of the experiment using xmlchange and namelist

**Time to learn**: 30 minutes

**NOTE**: Hands-on exercise is in the next module. You don't need to do anything yet.

---

<img src="./images/CESM2_Workspaces_Code.png"
         width="500"
         alt="CESM"
         align="right"
         style="horizontal-align:left;margin:10px 0px"/>
         
## CESM as a software  
- ~5 million lines of FORTRAN* source code and scripts (vs. Minecraft of 0.5 million in Java)
- You need to **build the source code** to create and run the executable.
- Multiple components communicating with each other through the coupler
- Download the released CESM from Github*
  - git clone -b release-cesm2.1.5 https://github.com/ESCOMP/CESM.git cesm2.1.5
  - cd cesm2.1.5
  - ./manage_externals/checkout_externals
- Read more about CESM [here](https://www.cesm.ucar.edu/)

*NOTE: [Why FORTRAN?](https://partee.io/2021/02/21/climate-model-response/)  
*NOTE: for this tutorial, please use the code in my workspace: `/glade/work/jiangzhu/paleocamp/cesm2.1.5`

---
<img src="./images/CESM2_Workspaces_Detail.png"
         width="750"
         alt="CESM"
         align="right"
         style="horizontal-align:left;margin:10px 0px"/>

## CESM Workspaces (assuming you run a company that builds new  electric cars)
1. CESM Code
  - "_Headquarter_", containing your intellectual properties, e.g., research/designing of EVs
  - FORTRAN, Python, & XML (Extensible Markup Language) files
2. Inputdata Directory
  - "_Supply Warehouse_", where you get the raw materials, e.g., steel and batteries
  - Netcdf data
3. Case Directory
  - "_Production Control Room_", where you give instructions, e.g., hire 100 workers and produce 5 EVs per day
  - Text, script, XML files
4. Build/Run Directory
  - "_Factory_", where workers assemble the EVs
  - Library, executables, netcdf files, ...
5. Archive Directory
  - "_Storage_", where you store the assembled EVs
  - Netcdf files, log files in text, ...



---
## Run a preindustrial control simulation in four steps

<div class="alert alert-info">

1. Create a new case `case01` in `~/cases` (takes space in CESM Code directory, the "_Headquarter_" and creates the "_Production Control Room_")
```bash
cd /glade/work/jiangzhu/paleocamp/cesm2.1.5/cime/scripts/
./create_newcase --case ~/cases/case01 --compset B1850 --res f19_g17 --project UAZN0035
```

2. Set up your case (takes space in Case directory, the "_Production Control Room_" and creates the "_Factory_")
```bash
cd ~cases/case01
./case.setup
```

3. Build your case (takes space in Case directory, the "_Production Control Room_" and designs the manufacturing workflow)
```bash
./case.build
```

4. submit your run (takes space in Case directory, the "_Production Control Room_" and starts the manufacturing)
```bash
./case.submit
```
</div>

### Further explanations on `create_newcase` 

The `create_newcase` command has three required inputs. _It creates your "Production Control Room"_.

1. `--case`: the name and directory path of your case (_your Production Control Room_)  
  - Setting case to `~/cases/case01` will make a simulation called `case01` and it will be located in `~/cases`
  - We use `case01` for simplicity, but here is the [CESM case naming conventions](https://www.cesm.ucar.edu/models/cesm2/naming-conventions)

2. `--compset`: the CESM component set that you’d like to use.
  - CESM2 components can be combined in numerous ways to carry out various scientific experiments. A particular mix of components, along with component-specific configuration and/or namelist settings is called a component set or `compset`.
  - `B1850` means fully coupled preindustrial control including active atmosphere, land, ocean, sea ice, etc.
  - `E1850` means coupled preindustrial control with active atmosphere and land but a slab ocean (no ocean dynamics).
  - `F1850` means atmosphere/land-only simulation with fixed sea-surface temperature and sea ice cover.
  - [Other supported compsets](https://docs.cesm.ucar.edu/models/cesm2/config/compsets.html). Or `$CESMROOT/cime/scripts/query_config --compsets`

3. `--res`: the resolution that you’d like to use.
  - `f19_g17`: ~2° atmosphere/land and ~1° ocean/sea ice.
  - `f09_g17`: ~1° atmosphere/land and ~1° ocean/sea ice
  - [Other supported grid/resolution](http://www.cesm.ucar.edu/models/cesm2/config/grids.html). Or `$CESMROOT/cime/scripts/query_config --compsets`

### Further explanations on `case.setup` 

`case.setup` creates the `$RUNDIR` and `$EXEROOT`, which is your _Factory_.  
`case.setup` also creates files like user_nl_cam, where users can customize component namelist settings.  
`case.setup` should be invoked in the $CASEROOT (_your Production Control Room_).  

### Further explanations on `case.build` 

`case.build` check and builds component model namelists and libraries, and the final executable.  
`case.build` _***may take 10 minutes!***_ 

Use the following to move the building job to a computing node (not the shared login node):  
`qcmd -- ./case.build`

Tips: if you first case.build fails somehow, run `./case.build --clean-all` first before trying to build the model again.

### Further explanations on `case.submit` 

`case.build` submit the job to the HPC batch job scheduler.  
`case.build` also submit the `case.st_archive` script to archive the model output (move output from your _Factory_ to _Storage_). This step is dependent on the successful completion of simulation.

---
## Check job status
`qstat` accesses the information in the Batch Job Scheduler to see the status of all jobs running on Derecho. To simplify the list down the `-u` option can be specified for a particular user.

`qstat` can be used in any directory on Derecho.

<div class="alert alert-info">
<strong>Result of running qstat with an active job running: </strong><br><br>

```bash
qstat -u $USER
```
<br>
    
<strong>Output: </strong><br>

```bash
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
704942.chadmin* $USER regular  run.b.day*   8284  16 576    --  12:00 R 00:00
704945.chadmin* $USER regular  st_archiv*    --    1   1    --  00:20 H   --  
```

</div>

Note, there may be a slight delay from submitting CESM until the jobs appear in the queue. Once the jobs have completed, they will disappear from the qstat command.

---
## Model output

If the model run is successful, the CESM netcdf output history files are automatically moved to a short-term Archive Directory (Your _Storage_; `$DOUT_S_ROOT`).  
**Notes:**
- If a model run was unsuccessful the output remains in the Run Directory (your _Factory_; `$RUNDIR`) and the short-term archive is not created.
- Both `$RUNDIR` and `$DOUT_S_ROOT` are in the NCAR HPC scratch space. This space is scrubbed, and files deleted after a number of days. Thus, it is a good idea to move your model output files from the short-term archive to a more permanent location as soon as you are able.

---
## More controls: XML change
Much of the control of the running of your CESM simulation is done through modifying the `env_run.xml` file, using the `xmlchange` command.


The **step** to modify the **xml** in ``$CASEROOT`` is:  
<div class="alert alert-info" style="text-align: left;">
./xmlchange VARIABLE=VALUE
</div>

For example, the following command changes the simulation length of a single submission to 1 month (the default setting is 5 days):

<div class="alert alert-info" style="text-align: left;">
./xmlchange STOP_OPTION=nmonths,STOP_N=1
</div>


- `STOP_OPTION` controls the units in which you are specifying the length of the simulation 
- and `STOP_N` is the length in those units.


You can check values of specific variables using:
<div class="alert alert-info" style="text-align: left;">
./xmlquery STOP_OPTION,STOP_N
</div>


You can find a listing of all the available XML variables along with their description in `env_run.xml` by running the following commands from your case directory
<div class="alert alert-info" style="text-align: left;">
./xmlquery --listall --file env_run.xml --description
</div>

Note: Typically, supercomputers have limits on how long you can 
run at once. For instance, some supercomputers have a wallclock 
limit of 12 hours in real time, meaning that you can only run 
continuously for 12 hours.
In that case, you would want to set `STOP_OPTION` and `STOP_N` to an appropriate length that can be completed within 12 hours of real time.

---
## More controls: namelist change
To control how the code is run (such as specific cloud parameters and atmospheric CO2 concentration), we modify the namelist settings.


The **steps** to modify the **namelists** in ``$CASEROOT`` are:
<div class="alert alert-info">
    
    - edit the ``user_nl_xxx`` files (use your favorite text editor, such as Nano, vi, etc.)
    
    - generate the namelists by running `./preview_namelists`.  
</div>


Complete documentation about the namelist variables can be found on the [CESM webpage.](https://www2.cesm.ucar.edu/models/cesm2/settings/current/)

---
## Summary
- CESM is a software primarily in FORTRAN and You need to build the executable.
- Like managing an EV company, you deal with multiple workplaces. What are they?
- Build and run a CESM case in four steps. What are they?
- How do you further customize the simulation?

---
## Optional advanced resources
* Check out the full yearly CESM Tutorial [here](https://ncar.github.io/CESM-Tutorial/README.html)
* [visualCaseGen](https://github.com/ESMCI/visualCaseGen): a GUI that runs on JupyterLab and guides the users through the process of creating CESM cases, e.g., choosing appropriate compsets and grids.